<table style="width: 100%; border-style: none;">
<tr style="border-style: none">
<td style="border-style: none; width: 1%; font-size: 16px">Institut f&uuml;r Theoretische Physik<br /> Universit&auml;t zu K&ouml;ln</td>
<td style="border-style: none; width: 1%; font-size: 16px">&nbsp;</td>
<td style="border-style: none; width: 1%; text-align: right; font-size: 16px">Prof. Dr. Simon Trebst<br />MSc. Carsten Bauer</td>
</tr>
</table>
<hr>
<h1 style="font-weight:bold; text-align: center; margin: 0px; padding:0px;">Computerphysik</h1>
<h1 style="font-weight:bold; text-align: center; margin: 0px; padding:0px;">Vorlesung &mdash; Programmiertechniken 5</h1>
<hr>
<h3 style="font-weight:bold; text-align: center; margin: 0px; padding:0px; margin-bottom: 20px;">Sommersemester 2020</h3>

**Website:** [http://www.thp.uni-koeln.de/trebst/Lectures/2020-CompPhys.shtml](http://www.thp.uni-koeln.de/trebst/Lectures/2020-CompPhys.shtml)

**Themen dieses Notebooks:** Differentialgleichungen in Julia, Endliche Machinenpräzision

## Differentialgleichung in Julia

Links: [DifferentialEquations.jl](https://github.com/JuliaDiffEq/DifferentialEquations.jl) ([Dokumentation](https://docs.juliadiffeq.org/latest/index.html))

### Beispiel Problem: Kohlenstoffzerfall

Der Zerfall von Kohlenstoff-14 folgt der folgenden gewöhnlichen Differentialgleichung erster Ordnung

$$ \frac{du(t)}{dt} = −c u(t) = f(u, p ,t)$$

wobei $u(t)$ die Kohlenstoffkonzentration darstellt und $c=5730$ Jahre die Halbwertszeit von C14 ist.

### Allgemeine Schritte um das Problem in Julia zu lösen

1. Problem definieren
2. Problem lösen
3. Lösung analysieren/visualisieren

## Schritt 1: Problem definieren

Das **Anfangswertproblem** ist charakteresiert durch
* einen Anfangswert $u_0$,
* eine Zeitspanne $t_{span}$ und
* die Differentialgleichung $\frac{du}{dt} = f(u,p,t)$.

## Schritt 2: Problem lösen

## Schritt 3: Problem analysieren/visualisieren

Das Lösungsobjekt `sol` ist eine Funktion, die automatisch zwischen den berechneten Funktionswerten interpoliert:

In [ ]:
u = u0 .* exp.(-5.730 .* sol.t) # analytische Lösung

In [ ]:
plot(sol.t, u, color="black", linewidth=2, linestyle="dashed", label="analytisch")
plot(sol.t, sol.u, color="red", alpha=.7, marker="o", label="numerisch")
legend()
ylabel("u(t)")
xlabel("t");

# 2. Differentialgleichungen höherer Ordnungen

### Beispiel Problem: Klassisches Pendel

$$
\ddot{\theta} + \frac{g}{L}{\sin(\theta)} = 0
$$

Analytisch lösen wir diese Differentialgleichung typischerweise in der "Kleine Winkel Näherung", d.h. $ sin(\theta) \approx \theta$, da wir sonst Elliptische Integrale erhalten die keine geschlossene analytische Lösung haben.

In Julia haben wir aber numerische Integratoren zur Hand! Wieso dann nicht die volle Differentialgleichung lösen?

$$
\begin{align}
\dot{u_1} &= u_2 \\
\dot{u_2} &= - \frac{g}{L}{\sin(u_1)}
\end{align}
$$

wobei $u_1 = \theta$ und $u_2 = \dot{\theta}$.

In [ ]:
plot(sol.t, getindex.(sol.u, 1), "bo-",);
xlabel("t")
ylabel("θ(t)");

In [ ]:
for u in sol.u
    polar(u[1], L, marker="o", linestyle="none", color="black")
end

# Rotate polar plot such that zero is "in the south"
gca().set_theta_zero_location("S")

# 3. Differentialgleichungssysteme: Der Lorenz-Attraktor

Für die Simulation eines einfachen **Wetter-Modells** hat der Meteorologe Edward Lorenz 1963 eine Beschreibung von 
Luftströmungen entworfen. Dazu hat er ein Gleichungssystem von drei gekoppelten Differentialgleichungen betrachtet:

$$
\begin{align}
\frac{d x}{d t} &= \sigma(y-x) \\
\frac{d y}{d t} &= x(\rho-z)-y \\
\frac{d z}{d t} &= x y-\beta z
\end{align}
$$

Im folgenden wollen wir dieses Differentialgleichungssytem lösen.

In [ ]:
σ = 10.0
ρ = 28.0
β = 8/3

params = [σ, ρ, β]

function lorenz!(du,u,p,t)
    σ, ρ, β = p
    du[1] = σ * (u[2] - u[1])
    du[2] = u[1] * (ρ - u[3]) - u[2]
    du[3] = u[1] * u[2] - β * u[3]
end

In [ ]:
u0 = [1.0; 0.0; 0.0]

tspan = (0.0, 100.0)

prob = ODEProblem(lorenz!, u0, tspan, params)

sol = solve(prob, saveat=0.01)

In [ ]:
ux = getindex.(sol.u, 1) # getindex(x, i) == x[i]
uy = getindex.(sol.u, 2)
uz = getindex.(sol.u, 3)

pygui(true) # activate external plotting window
plot3D(ux, uy, uz);

In [ ]:
pygui(false); # deactivate external plotting window

## Differentialgleichungen mit Messunsicherheiten

In [ ]:
sol = solve(problem, saveat=0.025)

# analytic solution
u = u0 .* exp.(-(5.730 ± 2) .* sol.t);

# plot solution
ts = getfield.(sol.t, :val)
solvals = getfield.(sol, :val)
solerrs = getfield.(sol, :err);

errorbar(ts, solvals, yerr=solerrs)
plot(ts, getfield.(u, :val), color="red", lw=2)
ylabel("u(t)")
xlabel("t");

Randkommentar: Das Beeindruckende ist hier, dass die Autoren von DifferentialEquations.jl und Measurements.jl [nie zusammengearbeitet haben](https://discourse.julialang.org/t/differentialequations-jl-and-measurements-jl/6350). Das Feature wurde durch Julias Konstruktionsprinzip "automatisch" erzeugt. Für Interessierte gibt es [hier](https://github.com/crstnbr/JuliaOulu20/blob/master/Day1/4_generic_programming.ipynb) mehr dazu.

# Endliche Machinenpräzision

Wie Objekte im Computer repräsentiert werden, d.h. wie sie in Nullen und Einsen kodiert werden, ist durch ihren Typ bestimmt. Wir können den Typ einer Variable mit Hilfer der Funktion `typeof` herausfinden.

Julia ist eine sogenannte <b>dynamisch typisierte Sprache</b>, was bedeutet, dass Objekte ihren Typ dynamisch je nach Kontext verändern können. Zu jedem Zeitpunkt hat jedes Objekt jedoch einen bestimmten Typ.

In [78]:
a = 3
println(typeof(a))

a = "test"
println(typeof(a))

Int64
String


Die herkömmlichen Datentypen wie `Int64` und `Float64` stellen Zahlen durch eine endliche Anzahl von 64 Bits (Einsen und Nullen) dar. Daraus folgt direkt, dass z.B. nicht jede Fließkommazahl exakt repräsentiert werden kann.

Ein weiterer Effekt der endlichen Repräsentation von Zahlen ist der sogenannt *Integer-Overflow*.

In den allermeisten Fällen ist die endliche Machinenpräzision kein Problem. Falls doch kann man, auf Kosten der Laufzeit, Datentypen mit beliebiger Präzision verwenden: